# Yelp API Restaurant Calls By GPS Coordinates

For each coordinate listed in grid_coordinates.csv, return first 1k restaurants found via yelp api and
extract data regarding:

- name
- address & zip
- coordinates
- rating
- review count
- price level
- category
- yelp id

Write data to restaurant_data_coord.csv

In [55]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import csv

# Import API key
from api_keys import api_key

### Perform API calls

- Dataframe from grid_coordinates.csv
- Build function that calls and writes restaurant data to new csv, given pair of coordinates
- Run list of coordinates through function

In [56]:
# Create dataframe from csv file
coords_df = pd.read_csv("Resources/grid_coordinates.csv", 
                        names=["Lat", "Lng"])
coords_df

,Lat,Lng
0,38.871657,-123.109843
1,38.871657,-123.017222
2,38.871657,-122.924601
3,38.871657,-122.831980


In [57]:
# Function returns up to 1000 restaurant listings for input coordinates
def get_restaurants(lat, lng, api_key):
    
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {"Authorization": "Bearer %s" % api_key}
    restaurant_data = []
    count = 0
    
    # Increases search return limit
    for offset in range(0, 1000, 50):
        
        # Set parameters and pass into API calls, radius 8046 meters = 5 miles
        params = {"term":"restaurants", "latitude":lat, "longitude":lng, "radius":8046, 
                  "limit":50, "offset":offset}
        req = requests.get(url, params=params, headers=headers)
        
        # Breaks if error occurs with search
        if req.status_code == 400:
            break
        elif req.status_code == 200:
            
            # Convert to json
            response = req.json()
            
            # Log history
            count += 1
            print(f"Now processing query set {count} of max 20")
            
            # Breaks if no further entries in query
            if response["businesses"] == []:
                break
            
            else:
                # Iterate through business results and extract data
                for biz in response["businesses"]:
                    
                    # Logic to replace missing price level data with NaN
                    if "price" not in biz:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], "", biz["categories"][0]["title"], 
                                                biz["id"]])

                    # Replace missing category data with empty string
                    elif biz["categories"] == []:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], "", biz["id"]])

                    else:
                        restaurant_data.append([str(lat) + ", " + str(lng), lat, lng, biz["name"], biz["coordinates"]["latitude"], 
                                                biz["coordinates"]["longitude"], biz["location"]["address1"], 
                                                biz["location"]["zip_code"], biz["rating"], 
                                                biz["review_count"], biz["price"], 
                                                biz["categories"][0]["title"], biz["id"]])

    # Write to csv
    with open('Resources/restaurant_data_coords.csv', 'a', encoding="utf-8") as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(restaurant_data)
    csvFile.close()
    
    # Returns total count of restaurants in coordinates
    return response["total"]


In [58]:
# Track # of coordinates processed
count = 0

# List to track total restaurants found in coordinate
totals_count = []

print("LOG HISTORY OF API CALLS:")
print("---------------------------")

# Loop thru list of coordinates
lat_coords = [lat for lat in coords_df["Lat"]]
lng_coords = [lng for lng in coords_df["Lng"]]

for index in range(len(lat_coords)):
                   
    #  Call get_restaurants fn and append to total_count list
    totals_count.append(get_restaurants(lat_coords[index], lng_coords[index] , api_key))
    
    # Print log history
    count += 1
    rem = len(lat_coords) - count
    print("-----------------------------------------")
    if count == len(lat_coords):
        print("Full list of coordinates processed!")
    elif count == len(lat_coords) - 1:
        print("Now getting results for final coordinates. Almost there!")
        print("-----------------------------------------")
    else:
        print(f"Data retrieval for coordinates complete")
        print(f"Getting results for next coordinates.. there are {rem} coordinates left")
        print("-----------------------------------------")

coords_df["Total # of Restaurants"] = totals_count

LOG HISTORY OF API CALLS:
---------------------------
Now processing query set 1 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 3 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
Now processing query set 2 of max 20
-----------------------------------------
Data retrieval for coordinates complete
Getting results for next coordinates.. there are 2 coordinates left
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Now getting results for final coordinates. Almost there!
-----------------------------------------
Now processing query set 1 of max 20
-----------------------------------------
Full list of coordinates processed!


In [59]:
# Show cities_df with updated total # of restaurants
coords_df.head()

,Lat,Lng,Total # of Restaurants
0,38.871657,-123.109843,0
1,38.871657,-123.017222,23
2,38.871657,-122.924601,0
3,38.871657,-122.831980,0


### Create Dataframe

- Read csv file
- Clean data

In [60]:
# Read csv file
restaurants_df = pd.read_csv("Resources/restaurant_data_coords.csv", encoding = "ISO-8859-1", 
                            names=["search_ID", "Latitude_search", "Longitude_search", "Name", "Lat_Restaurant", "Lng_Restaurant", "Address", "Zip", 
                                   "Rating", "# of Reviews", "Price Level", "Category", "Yelp ID"])

# Replace NaN entries with blank string
restaurants_df = restaurants_df.fillna('')
restaurants_df.head()

,search_ID,Latitude_search,Longitude_search,Name,Lat_Restaurant,Lng_Restaurant,Address,Zip,Rating,# of Reviews,Price Level,Category,Yelp ID
0,"38.871657, -123.017222",38.871657,-123.017222,Trading Post,38.805104,-123.016866,102 S Cloverdale Blvd,95425,4.0,128,$$,Bakeries,6o26tulocbwnkZx89EVHkA
1,"38.871657, -123.017222",38.871657,-123.017222,Hamburger Ranch & Bar-B-Que,38.817818,-123.023628,31195 N Redwood Hwy,95425,4.0,381,$$,Barbeque,0Dj4fW3J3DJzrI51PLadRA
2,"38.871657, -123.017222",38.871657,-123.017222,Railroad Station Bar and Grill,38.803078,-123.015378,236 S Cloverdale Blvd,95425,4.0,223,$$,American (New),slg-wyyA57sZSMWmNshl-Q
3,"38.871657, -123.017222",38.871657,-123.017222,Zini's Diner,38.791995,-123.016825,796 S Cloverdale Blvd,95425,4.0,108,$$,American (Traditional),qqpHX6ksqim3Sgm94zkTLA
4,"38.871657, -123.017222",38.871657,-123.017222,Cloverdale Ale Company,38.805637,-123.015944,131 E 1st St,95425,4.0,21,$$,Pubs,wAbWk-do6kAUt3V9z8f9Wg


In [159]:
### Clean data, how to treat missing address, zip, price level, food trucks/mobile, low review counts
### Groupby on df